In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

import fiona
import folium
from shapely import wkt
from shapely.geometry import shape
import ast

import json
from loguru import logger
from pathlib import Path

from geowrangler.datasets import geofabrik

import sys

sys.path.append("../../../")  # include parent directory
from src.vector_utils import *

In [5]:
# data directories
DATA_DIR = Path("../../../data/")
SRC_DIR = Path("../../../src/")
ADMIN_FPATH = DATA_DIR / "01-admin-bounds"
RAW_FPATH = DATA_DIR / "02-raw"
PROCESSED_FPATH = DATA_DIR / "03-processed"
OUTPUT_FPATH = DATA_DIR / "04-output"
GIS_FPATH = DATA_DIR / "05-gis"

In [6]:
# POI types of OSM
# tags taken from https://wiki.openstreetmap.org/wiki/Map_features
poi_file = Path(SRC_DIR / "osm_poi_categories.json")

with open(poi_file, "r") as f:
    data = json.load(f)

POI_TYPES_NEEDED = []
for category in data:
    POI_TYPES_NEEDED += data[category]

In [9]:
# Load the administrative boundaries for the 12 cities
admin_bounds_gdf = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
admin_bounds_gdf.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


# OSM Download

In [7]:
osm = geofabrik.OsmDataManager(cache_dir="../../../data/02-raw/")

## Download OSM 2022

In [8]:
osm_pois_2022 = osm.load_pois(region="philippines", year="2022", use_cache=True)
osm_pois_2022.head(2)

2023-05-29 17:03:37.349 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:128 - OSM Data: Cached data available for philippines at ../../../data/02-raw/osm/philippines-220101-free.shp.zip? True
2023-05-29 17:03:37.350 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:222 - OSM POIs for philippines and year 2022 being loaded from ../../../data/02-raw/osm/philippines-220101-free.shp.zip


,osm_id,code,fclass,name,geometry
0,21717820,2907,camera_surveillance,None,POINT (121.02120 14.57608)
1,21717872,2722,museum,Ayala Museum,POINT (121.02324 14.55358)


In [ ]:
osm_pois_2022["fclass"].unique()

## Generate POI features

Use the `add_osm_poi_features` util from the `src` folder. This is just a copy of the [povertymapping util](https://github.com/thinkingmachines/unicef-ai4d-poverty-mapping/blob/main/povertymapping/osm.py). Copy pasted it because the actual function is not in geowrangler yet due to some blockers.

POI features to calculate for each admin bounds and POI type:
1. POI count
2. Distance to nearest POI

In [10]:
osm_poi_feats_gdf = add_osm_poi_features(
    admin_bounds_gdf,
    "philippines",
    "2022",
    osm,
    use_cache=True,
    poi_types=POI_TYPES_NEEDED,
)
osm_poi_feats_gdf.head(2)

2023-05-29 17:08:31.283 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:203 - OSM POIs for philippines and year 2022 found in cache.


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,osm_year,...,osm_poi_toilet_count,osm_poi_toilet_nearest,osm_poi_recycling_count,osm_poi_recycling_nearest,osm_poi_waste_basket_count,osm_poi_waste_basket_nearest,osm_poi_wastewater_plant_count,osm_poi_wastewater_plant_nearest,osm_poi_waste_transfer_station_count,osm_poi_waste_transfer_station_nearest
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",2022,...,0.0,5993.655956,0.0,2486.499185,0.0,10000.0,0.0,10000.0,0.0,10000.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2022,...,0.0,3736.276540,0.0,79.395847,0.0,10000.0,0.0,10000.0,0.0,10000.0


In [11]:
osm_poi_feats_gdf = osm_poi_feats_gdf.sort_values(by="poi_count", ascending=False)
# osm_poi_feats_gdf.to_file(PROCESSED_FPATH / "osm_poi_features.gpkg", driver="GPKG")

In [12]:
remove_pcode = ["ADM1_PCODE", "ADM2_PCODE", "ADM3_PCODE", "ADM4_PCODE"]
check_osm_poi_feats = osm_poi_feats_gdf.copy()
check_osm_poi_feats = check_osm_poi_feats.loc[
    :, ~check_osm_poi_feats.columns.isin(remove_pcode)
]

In [13]:
def get_count_cols(df):
    count_cols = [x for x in df.columns[df.columns.str.contains("_count")]]
    return count_cols


def get_nearest_cols(df):
    nearest_cols = [x for x in df.columns[df.columns.str.contains("_nearest")]]
    return nearest_cols

In [14]:
admin_cols = ["ADM1_EN", "ADM2_EN", "ADM3_EN", "ADM4_EN", "geometry"]

# get columns with "_count" suffix
count_cols, nearest_cols = get_count_cols(check_osm_poi_feats), get_nearest_cols(
    check_osm_poi_feats
)

poi_counts = check_osm_poi_feats[admin_cols + count_cols]
poi_nearest = check_osm_poi_feats[admin_cols + nearest_cols]

Check POI counts aggregated for each city. 

Davao City, Iloilo City, and Muntinlupa City have high POI counts, while Palayan City and Navotas City have very low POI counts (both are less than a hundred).

In [15]:
city_stats_counts = poi_counts.groupby(["ADM3_EN"]).sum()
city_stats_counts.sort_values(by="poi_count", ascending=False)

/tmp/ipykernel_6659/863534221.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  city_stats_counts = poi_counts.groupby(["ADM3_EN"]).sum()


,poi_count,osm_poi_clinic_count,osm_poi_dentist_count,osm_poi_doctors_count,osm_poi_hospital_count,osm_poi_optician_count,osm_poi_pharmacy_count,osm_poi_atm_count,osm_poi_bank_count,osm_poi_college_count,...,osm_poi_water_mill_count,osm_poi_water_tower_count,osm_poi_water_works_count,osm_poi_water_well_count,osm_poi_sanitary_dump_station_count,osm_poi_toilet_count,osm_poi_recycling_count,osm_poi_waste_basket_count,osm_poi_wastewater_plant_count,osm_poi_waste_transfer_station_count
ADM3_EN,,,,,,,,,,,,,,,,,,,,,
Davao City,3424.0,0.0,13.0,40.0,4.0,2.0,130.0,70.0,216.0,30.0,...,0.0,6.0,1.0,2.0,0.0,32.0,4.0,4.0,0.0,0.0
Iloilo City,1661.0,0.0,2.0,8.0,3.0,3.0,71.0,32.0,103.0,13.0,...,0.0,1.0,0.0,2.0,0.0,15.0,0.0,0.0,0.0,0.0
City of Muntinlupa,1014.0,0.0,12.0,8.0,3.0,7.0,41.0,18.0,110.0,3.0,...,0.0,35.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0
Cagayan de Oro City,987.0,0.0,10.0,3.0,3.0,0.0,54.0,16.0,69.0,9.0,...,0.0,8.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
City of Mandaluyong,968.0,0.0,15.0,5.0,0.0,1.0,41.0,35.0,98.0,5.0,...,0.0,2.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0
Tacloban City,781.0,0.0,5.0,5.0,2.0,3.0,29.0,12.0,33.0,6.0,...,0.0,0.0,0.0,29.0,0.0,1.0,1.0,0.0,0.0,0.0
Dagupan City,751.0,0.0,12.0,4.0,1.0,5.0,49.0,3.0,48.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Zamboanga City,537.0,0.0,3.0,2.0,1.0,0.0,47.0,57.0,29.0,4.0,...,0.0,5.0,2.0,5.0,0.0,9.0,0.0,0.0,1.0,0.0
Mandaue City,531.0,0.0,1.0,6.0,1.0,0.0,36.0,6.0,54.0,6.0,...,0.0,1.0,0.0,4.0,0.0,7.0,1.0,0.0,0.0,0.0


# Download remaining years

In [ ]:
osm_pois_2021 = osm.load_pois(region="philippines", year="2021", use_cache=True)
osm_pois_2021.head(2)

In [ ]:
osm_pois_2020 = osm.load_pois(region="philippines", year="2020", use_cache=True)
osm_pois_2020.head(2)

In [ ]:
osm_pois_2019 = osm.load_pois(region="philippines", year="2019", use_cache=True)
osm_pois_2019.head(2)

In [ ]:
osm_pois_2018 = osm.load_pois(region="philippines", year="2018", use_cache=True)
osm_pois_2018.head(2)

In [ ]:
osm_pois_2017 = osm.load_pois(region="philippines", year="2017", use_cache=True)
osm_pois_2017.head(2)

In [ ]:
osm_pois_2016 = osm.load_pois(region="philippines", year="2016", use_cache=True)
osm_pois_2016.head(2)

In [ ]:
osm_pois_2015 = osm.load_pois(region="philippines", year="2015", use_cache=True)
osm_pois_2015.head(2)

In [7]:
osm_pois_2014 = osm.load_pois(region="philippines", year="2014", use_cache=True)
osm_pois_2014.head(2)

2023-05-16 17:05:00.555 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:128 - OSM Data: Cached data available for philippines at ../../data/02-raw/osm/philippines-140101-free.shp.zip? False
2023-05-16 17:05:01.839 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:142 - OSM Data: Re-initializing OSM region zip file at ../../data/02-raw/osm/philippines-140101-free.shp.zip...
2023-05-16 17:05:01.840 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:149 - OSM Data: Downloading Geofabrik in ../../data/02-raw/osm/philippines-140101-free.shp.zip...


2023-05-16 17:09:55.297 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:165 - OSM Data: Successfully downloaded and cached OSM data for philippines and 2014 at ../../data/02-raw/osm/philippines-140101-free.shp.zip!
2023-05-16 17:09:55.298 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:222 - OSM POIs for philippines and year 2014 being loaded from ../../data/02-raw/osm/philippines-140101-free.shp.zip


,osm_id,code,fclass,name,geometry
0,10970809,2099,public_building,Sagada Municipal Hall,POINT (120.90073 17.08419)
1,14401658,2742,viewpoint,East Point,POINT (120.62030 14.38557)


Downloading past `2014` will result in an error because data is unavailable.

In [14]:
osm_pois_2000 = osm.load_pois(region="philippines", year="2000", use_cache=True)
osm_pois_2000.head(2)

2023-05-18 13:21:35.786 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:128 - OSM Data: Cached data available for philippines at ../../data/02-raw/osm/philippines-000101-free.shp.zip? False
2023-05-18 13:21:36.871 | WARNING  | geowrangler.datasets.geofabrik:download_osm_region_data:139 - OSM data for philippines and year 2000 is not available


AttributeError: 'NoneType' object has no attribute 'head'